## Pipeline обработки запроса

### Загрузка данных и функций (достаточно выполнить 1 раз)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from utils import setup_environment, load_data, setup_logging, save_results_to_excel

from retriever import retriever
from planner import planner
from grounder import grounder
from executor import executor

from schemas import RetrieverOut, PlannerOut

from openai import OpenAI
from config import PipelineConfig


api_key, db_path = setup_environment()
print(f"Используемый API KEY: {api_key[:3]}...{api_key[-3:]}")

db = load_data(db_path)

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key,
)

### Настройка используемых моделей и их температур

Если нужно поменять модель, это делается здесь

In [ ]:
PPL_cfg = PipelineConfig.setup(
    df=db[db["wave"] == "2025-03"],
    client=client,
    # параметры ретривера
    retriever_params={
        "model": "x-ai/grok-4.1-fast",
        # "model": "alibaba/tongyi-deepresearch-30b-a3b:free",
        # "model": "deepseek/deepseek-chat-v3.1",
        "temperature": 0.8,
        "reasoning_effort": "low"
    },
    # параметры планировщика
    planner_params={
        "model": "x-ai/grok-4.1-fast",
        # "model": "openai/gpt-oss-20b:free",
        # "model": "alibaba/tongyi-deepresearch-30b-a3b:free",
        # "model": "deepseek/deepseek-chat-v3.1",
        # "max_tokens": 25_000,
        "temperature": 0.2,
        # "reasoning_effort": "low"
    }
)

### Logger

Настройка логирования и директорий сохранения файлов

In [ ]:
run_dir = setup_logging(mode="DEBUG", log_dir="logs")
# run_dir = r"E:\Work\IvRAG\logs\run_2025-11-26_16-26-21"

retriever_out_path = os.path.join(run_dir, "retrieved.json")
dreamer_out_path = os.path.join(run_dir, "dreamed.json")
planner_out_path = os.path.join(run_dir, "planned.json")

### Retriever

Извлекает релевантные вопросы на основе запроса пользователя и всего набора вопросов

Пользовательский запрос

In [ ]:
# user_query = "Я хочу посчитать индекс потребительской уверенности по потребителям из москвы"
# user_query = "Я хочу посчитать размер средних сбережений и норму сбережений среди тех, у кого они есть"
# user_query = "Я хочу посчитать долю взаимопроникновения клиентов Чижика и Пятерочки"
# user_query = "Мне нужны все вопросы, связанные с автотранспортом (сроки владения, предпочтения по маркам, планы по покупке и пр.)"
# user_query = "Посчитай размер средних сбережений и норму сбережений среди тех, у кого они есть, среди жителей Москвы"
# user_query = "Мне нужно посчитать средний доход потребителей старше 30 лет"
# user_query = "Выведи все вопросы связанные с туризмом"
# user_query = "Хочу посчитать среднюю зарплату жителей москвы"
user_query = "Мне нужны сводные таблицы по всем вопросы, связанные с отраслью туризма"

Обращение к LLM

> ! Возможны проблемы парсинга ответов

In [ ]:
# Сохранение, чтобы не делать 1 и тот же запрос кучу раз:
# можно 1 раз сохранить, а далее только читать

retriever_out = retriever(user_query, PPL_cfg)
retriever_out.save(retriever_out_path)

Можно посмотреть reasoning, если модель его поддерживает и он был указан в параметрах

In [ ]:
# print(retriever_out.reasoning)

In [ ]:
retriever_out = RetrieverOut.load(retriever_out_path)
PPL_cfg.update_context(retriever_out)

print(retriever_out)

### Planner

Строит план на основе пользовательского запроса и набора релевантных ответов, к которым подмешиваются их ответы

В план включаются команды из [`capability_spec.py`](./capability_spec.py)

In [ ]:
# Сохранение, чтобы не делать 1 и тот же запрос кучу раз:
# можно 1 раз сохранить, а далее только читать

planner_out = planner(user_query, PPL_cfg)
planner_out.save(planner_out_path)

In [ ]:
planner_out = PlannerOut.load(planner_out_path)
print(planner_out)

### Grounder

Привязка шагов плана к имеющимся функциям (из [`operations.py`](./operations.py))

In [ ]:
grounder_out = grounder(planner_out)

### Executor

Валидирует план, осуществляет топологическую сортировку, выполняет последовательность шагов

In [ ]:
ctx = {"dataset": db}
ctx, prov = executor(grounder_out, ctx)

Сохраняем полученный контекст в excel

In [ ]:
save_results_to_excel(ctx, prov, f"{run_dir}\\results.xlsx")